In [2]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import scipy
from fitTools import AnalysisFunctions as af

# logical t2

In [3]:
dataDir = 'G:\\Shared drives\\CavityCooling\\data\\diss08_07A\\20230317\\T2wFFL'
filename = 'T2wFFL_fflFreq=6d6GHz_DA=24dB_navg=5000_9.h5'
f =  h5py.File(f'{dataDir}\\{filename}', 'r') 
print(f.keys())
key = list(f.keys())[0]
print(key)


<KeysViewHDF5 ['sweep_ffl_amp_19:31:47']>
sweep_ffl_amp_19:31:47


In [4]:
num_of_entries = len(f[key].keys())
num_of_entries

101

In [ ]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

def decay(x,amp,tau,offset):
    return amp*np.exp(-x/tau)+offset

In [ ]:
mag_array =[]
time_array = []
t2 = np.zeros((num_of_entries, ))
t2_err = np.zeros((num_of_entries, ))

for i in range(num_of_entries):
    entry0 = list(f[key].keys())[i]
    time = np.array(f[key][entry0]['t'])
    data = np.array(f[key][entry0]['I']) + 1j * np.array(f[key][entry0]['Q'])
    mag = abs(data) * 1e3
    mag_array.append(mag)
    time_array.append(time)
    tau = 0.2
    amp = abs(mag[0] - mag[-1])
    offset = 0

    p0 = [amp,tau,offset]
    fitFunction = decay
    fitted_pars, covar = scipy.optimize.curve_fit(fitFunction, time, mag,p0=p0,method='trf',xtol=1e-12,maxfev=40e3)
    error = np.sqrt(abs(np.diag(covar)))
    t1[i] = fitted_pars[1]
    t1_err[i] = error[1]
    if time[-1] > 1.75:
        cut = np.argmin(abs(time - 1.75))
        time = time[:cut]
        mag = mag[:cut]
    # 
    plt.plot(time, mag ,'x',color = cmap[i])
    if i is not 5:
        label='drive amp='+entry0.split(' = ')[1] + ', '
    else:
        label = 'drive off, '
    plt.plot(time, decay(time,fitted_pars[0], fitted_pars[1], fitted_pars[2]),'-',color =cmap[i],label=label+r'$\tau$'+'={%d}'%(fitted_pars[1]*1e3)+r'$\pm$'+'{%d} ns'%(t1_err[i]*1e3))

# logical t1